# Decision tree from scratch

In [31]:
import numpy as np
from sklearn.metrics import accuracy_score

In [27]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y, depth=0):
        n_samples, n_features = X.shape
        unique_classes = np.unique(y)
        
        if (len(unique_classes) == 1) or (depth == self.max_depth):
            return Node(None, None, None, None, unique_classes[0])
        
        feature_idx, threshold = self.find_best_split(X, y)
        left_indices, right_indices = self.split_data(X, feature_idx, threshold)
        
        if len(left_indices) == 0 or len(right_indices) == 0:
            return Node(None, None, None, None, unique_classes[np.argmax(np.bincount(y))])
        
        left_subtree = self.fit(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self.fit(X[right_indices], y[right_indices], depth + 1)
        
        # Set the root node if this is the initial call
        if depth == 0:
            self.root = Node(feature_idx, threshold, left_subtree, right_subtree, None)
        
        return Node(feature_idx, threshold, left_subtree, right_subtree, None)
    
    def predict(self, X):
        predictions = []
        for x in X:
            node = self.root
            while node.value is None:
                if x[node.feature_idx] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
            predictions.append(node.value)
        return predictions
        
    def find_best_split(self, X, y):
        n_samples, n_features = X.shape
        if n_samples <= 1:
            return None, None
        
        y_sorted = np.sort(y)
        splits = (y_sorted[1:] + y_sorted[:-1]) / 2.0
        
        best_gini = 1
        best_split = None
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices, right_indices = self.split_data(X, feature, threshold)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gini = self.calculate_gini(y[left_indices], y[right_indices])
                if gini < best_gini:
                    best_gini = gini
                    best_split = (feature, threshold)
        
        return best_split

    def split_data(self, X, feature_idx, threshold):
        left_indices = np.where(X[:, feature_idx] <= threshold)[0]
        right_indices = np.where(X[:, feature_idx] > threshold)[0]
        return left_indices, right_indices

    def calculate_gini(self, left_y, right_y):
        total_samples = len(left_y) + len(right_y)
        p_left = len(left_y) / total_samples
        p_right = len(right_y) / total_samples
        
        gini_left = 1 - sum((np.sum(left_y == c) / len(left_y))**2 for c in np.unique(left_y))
        gini_right = 1 - sum((np.sum(right_y == c) / len(right_y))**2 for c in np.unique(right_y))
        
        gini = p_left * gini_left + p_right * gini_right
        return gini

In [28]:
class Node:
    def __init__(self, feature_idx, threshold, left, right, value):
        self.feature_idx = feature_idx
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

In [29]:
X = np.array([[2, 3], [5, 6], [7, 8], [9, 10], [11, 12]])
y = np.array([0, 1, 1, 0, 0])

In [32]:
tree = DecisionTree(max_depth=2)
tree.fit(X, y)

my_predictions = tree.predict(X)
my_accuracy = accuracy_score(y, my_predictions)
print("Custom DecisionTree Accuracy:", my_accuracy)

Custom DecisionTree Accuracy: 1.0
